In [1]:
!mkdir -p sample_output/out_demog

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import json
import sparse
import pandas as pd
import numpy as np
import joblib

In [4]:
df_static = pd.read_csv('sample_input/demog.csv').set_index('hosp_id')

In [5]:
discretization_bins = json.load(open('metadata/out_demog/discretization.json', 'r'))
feature_names = json.load(open('metadata/out_demog/s_all.feature_names.json', 'r'))

In [6]:
discretization_bins

{'age_value': [18.0, 47.0, 59.0, 67.0, 76.0, 102.0],
 'sex_value': None,
 'race_value': None}

In [7]:
cols_useful = list(discretization_bins.keys())

In [8]:
cols_useful

['age_value', 'sex_value', 'race_value']

In [9]:
df_static

,age_value,sex_value,race_value
hosp_id,,,
90000,50,M,African American
90001,24,F,Caucasian


In [10]:
df_static = df_static[cols_useful]

In [11]:
from steps_2 import *
from helpers import *

In [12]:
df = df_static

In [13]:
df_by_cols = [df[col] for col in df.columns]

In [14]:
out_0 = [smart_qcut_bins((x, discretization_bins[x.name])) for x in tqdm(df_by_cols)]

100%|██████████| 3/3 [00:00<00:00, 145.03it/s]


In [15]:
cols_data, dis_bins = zip(*out_0)

In [16]:
out = [dummify(z) for z in tqdm(cols_data)]

100%|██████████| 3/3 [00:00<00:00, 406.73it/s]


In [17]:
df_features = pd.concat(out, axis=1).sort_index(axis=1)

In [18]:
# Drop any values='missing' for now
df_features = df_features.loc[:, [col for col in df_features.columns if 'missing' not in col]]

In [19]:
# Make sure same features
set(df_features.columns) <= set(feature_names)

True

In [20]:
df_features = df_features.reindex(columns=feature_names, fill_value=0)

In [21]:
df_features

,"age_value_(17.999, 47.0]","age_value_(47.0, 59.0]","age_value_(59.0, 67.0]","age_value_(67.0, 76.0]","age_value_(76.0, 102.0]",race_value_African American,race_value_American Indian or Alaska Native,race_value_Asian,race_value_Caucasian,race_value_Native Hawaiian and Other Pacific Islander,race_value_Other,race_value_Patient Refused,race_value_Unknown,sex_value_F,sex_value_M
hosp_id,,,,,,,,,,,,,,,
90000,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1
90001,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [22]:
sdf = df_features.astype(pd.SparseDtype(int, fill_value=0))
X_all = sparse.COO(sdf.sparse.to_coo())

In [23]:
sparse.save_npz('sample_output/out_demog/X_all.npz', X_all)
df_features[[]].to_csv('sample_output/out_demog/X_all.IDs.csv')
# json.dump(list(X_all_feature_names), open('out_meds/X_all.feature_names.json', 'w'), sort_keys=True)
# json.dump(discretization_bins, open('out_meds/discretization.json', 'w'))

In [24]:
import sparse, json

In [25]:
s = sparse.load_npz('sample_output/out_demog/X_all.npz').todense()

In [26]:
s.shape

(2, 15)

In [27]:
cols = feature_names
df_s = pd.DataFrame(s, columns=cols)
df_s.index = df_static.index

In [28]:
df_s.to_csv('sample_output/out_demog/static-features.csv')

In [29]:
df_s

,"age_value_(17.999, 47.0]","age_value_(47.0, 59.0]","age_value_(59.0, 67.0]","age_value_(67.0, 76.0]","age_value_(76.0, 102.0]",race_value_African American,race_value_American Indian or Alaska Native,race_value_Asian,race_value_Caucasian,race_value_Native Hawaiian and Other Pacific Islander,race_value_Other,race_value_Patient Refused,race_value_Unknown,sex_value_F,sex_value_M
hosp_id,,,,,,,,,,,,,,,
90000,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1
90001,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0
